<a href="https://colab.research.google.com/github/astuart06/Colab_dev/blob/master/astroABC_Vicsek2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ABC Vicsek model and MCMCMH
import os, sys
import numpy as np
from getpass import getpass

import matplotlib.pyplot as plt
plt.style.use('ggplot') 
plt.style.use('seaborn-paper') 
plt.style.use('seaborn-whitegrid') 

from IPython import display

from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib.collections as collections
%matplotlib inline
from itertools import product

#! pip install sobol_seq
!pip install mpi4py

# code for running in Colab
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    user = input('enter GitHub username: ')
    password = getpass('password: ')
    os.environ['git_user'] = user + ':' + password    
    
    !git clone https://$git_user@github.com/drraskell/astroABC  
    !git clone https://$git_user@github.com/ctorney/multiscale-abc-inference    
    
    sys.path.append('multiscale-abc-inference')
    sys.path.append('astroABC')

except Exception:
    pass


%matplotlib inline
import scipy
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from scipy.stats import skewnorm
import math

import os
if os.path.exists("abc_pmc_output_1param.txt"):
  os.remove("abc_pmc_output_1param.txt")

from scipy import interpolate
from scipy.interpolate import interp1d
from simulations import vicsek


#from distance_calc import DistanceCalc   
#from bin_data import *

enter GitHub username: drraskell
password: ··········
Cloning into 'astroABC'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.
Cloning into 'multiscale-abc-inference'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 413 (delta 137), reused 44 (delta 12), pack-reused 160
Receiving objects: 100% (413/413), 27.49 MiB | 18.86 MiB/s, done.
Resolving deltas: 100% (203/203), done.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import astroabc

In [0]:
#measures the difference between data and simulation
def my_dist(sumstat_data,sumstat_sim):
  return np.sum(np.abs(sumstat_data-sumstat_sim))   #can include std of data here

In [0]:
nparam = 1 #2
npart = 100 #200 #number of particles/walkers
niter = 1  #number of iterations
tlevels = [0.056629*20,20*0.056629*(0.9**(niter-1))] #cov of data repeats....do we want to remove implausible points too? 
#[0.1,0.005] #maximum,minimum tolerance

prop={'tol_type':'exp',"verbose":1,'adapt_t':True,
      'threshold':75,'pert_kernel':2,'variance_method':0,
      'dist_type': 'user','dfunc':my_dist, 'restart':"restart_test.txt", \
      'outfile':"abc_pmc_output_"+str(nparam)+"param.txt",'mpi':False,
      'mp':True,'num_proc':2, 'from_restart':False}

In [0]:
# priorname  = ["normal","normal"]    
# hyperp = [[0,100], [0,100]] #means and stds
# prior = list(zip(priorname,hyperp))
# print(prior)

In [0]:

# priorname  = ["uniform", "uniform"]     #["normal","normal"]    
# hyperp = [[1.8,2.5],[0,1]]             # [[0,100], [0,100]] #means and stds
prior = [('uniform',[1.8,2.5])]   #,('uniform,[0,1]')]   #[('uniform',[1.8,2.5]),('uniform,[0,1]')] #perhaps

In [0]:
#general parameters:
sz=1
timesteps=100
burnin=0
N = 1000
vs=0.01
repeat = 1
lr = 0.1
vs = 0.01

In [0]:
#forward simulation bit:


def ABCsimulation(params): #param = [om, w0] 
    simulation_cls = vicsek.vicsek_model(N,timesteps=timesteps,discard=burnin,L=sz,repeat=repeat, save_interval=1,disable_progress=True)

    simulation_cls.run_sim(params,lr, vs)
    sim_output, _, _ = simulation_cls.get_macro_states()

    sim_output = np.reshape(sim_output,(repeat,timesteps-1-burnin))
    end_output = sim_output[:,-1]
    print(np.shape(end_output))
    avgend = np.mean(end_output)

    return avgend



    # if param[0] < 1.8 or param[0] > 2.5:
    # #if param[0] < 0.0 or param[0] > 1.0:
    #     return [None]*len(zbins)
    # else:
    #     #model_1_class = DistanceCalc(param[0],0,1-param[0],0,[param[1],0],0.7)  #om,ok,ol,wmodel,de_params,h0

    #     data_abc = np.zeros(len(zbins))
    #     for i in range(len(zbins)):
    #             data_abc[i] = model_1_class.mu(zbins[i]) + skewnorm.rvs(a, loc=e, scale=w, size=1)
  

In [0]:
#make the data:
data_eta = 2.2
data = ABCsimulation(data_eta)

(1,)


In [0]:
data

0.264956533908844

In [0]:
def ABCsimulation2(params): 
    simulation_cls = vicsek.vicsek_model(N,timesteps=timesteps,discard=burnin,L=sz,repeat=repeat, save_interval=1,disable_progress=True)
    
    simulation_cls.run_sim(params[0],lr, vs)  
    sim_output, _, _ = simulation_cls.get_macro_states()

    sim_output = np.reshape(sim_output,(repeat,timesteps-1-burnin))
    end_output = sim_output[:,-1]
    print(np.shape(end_output))
    avgend = np.mean(end_output)

    return avgend

In [0]:
#the astroabc bit:
sampler = astroabc.ABC_class(nparam,npart,data,tlevels,niter,prior,**prop)
output = sampler.sample(ABCsimulation2)


	 	
	 ########################     astroABC     ########################	
	 	
	 Npart=100 	 numt=1 	 tol=[1.1326,1.1326] exp
	 Priors= [('uniform', [1.8, 2.5])]


In [0]:
import pandas as pd

dataout = pd.read_csv('abc_pmc_output_1param.txt', header = None, sep =' \t', skiprows =1, decimal='.' )
print(dataout[0])
np.shape(dataout)
#first column = value , second column = distance, third column = weighting 

#curve = plt.hist(dataout[0],bins=[2.0,2.05,2.1,2.15,2.2,2.25])
import seaborn as sns, numpy as np

sns.set(); 
sns.set_style(style='white')
ax1 = sns.distplot(dataout[0],bins=20)
ax1.set(xlabel='$\eta$', ylabel='posterior')


fit = ax1.get_lines()[0].get_data() # Getting the data from the plotted line
xfit, yfit = fit[0], fit[1]


In [0]:
def mcmcmh(ystest,ystd,z0, sigma, mineta,maxeta,leneta, n_samples, burn_in, m): 
    
    samples = np.zeros(n_samples)
    zmesh = np.linspace(mineta,maxeta,leneta)
    
    # Store initial value
    samples[0] = z0
    z = z0 

    f = interpolate.interp1d(zmesh,ystest)

    l_cur = f(z)

    if l_cur=={}:
        l_cur = - np.inf   #if implausible, set log likelihood to be -inf

    # Counter
    iter = 0
    # Total number of iterations to make to achieve desired number of samples
    iters = (n_samples * m) + burn_in

    # Sample outside the for loop
    innov = np.random.normal(loc=0, scale=sigma, size=iters)
    u = np.random.rand(iters)   #/3 +0.66

    while iter < iters:
        # Random walk innovation on z
        cand = z + innov[iter]
        
        l_cand = f(cand)
        
        if l_cand=={}:
            l_cand = - np.inf   #if implausible, set log likelihood to be -inf if input is log likelihood
        
        # Accept or reject candidate
        #if (l_cand / l_cur) > u[iter]:
        if np.exp(l_cand-l_cur) >u[iter]:
            z = cand
            l_cur = l_cand

        # Only keep iterations after burn-in and for every m-th iteration
        if iter > burn_in and iter % m == 0:
            samples[(iter - burn_in) // m] = z

        iter += 1       

    return samples

In [0]:
mcmcinput = yfit.flatten()   
mcmcstd = yfit.flatten() #not used

sobol_range = xfit

#mcmcmh_old.mcmcmh
samples = mcmcmh(np.log(mcmcinput),mcmcstd,np.random.choice(sobol_range).flatten(), 0.001, np.min(sobol_range),np.max(sobol_range),len(xfit), n_samples=10**6, burn_in=1000, m=1)


In [0]:
# plt.hist(samples,bins=100,density=True,range=())
# plt.axvline(data_eta, color='k')
# plt.xlabel('$/eta$')
# plt.show()
import seaborn as sns
sns.set(); 
sns.set_style(style='white')
ax = sns.distplot(samples,bins=100, hist_kws={"range": [1.5,2.9]})

ax.axvline(data_eta, color='k')
plt.xlabel('$\eta$')

In [0]:
print(dataout[1])